In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS, analyzer='word', binary=True, min_df = 10, max_df =0.4)
vectorizer.fit(train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.4, max_features=None, min_df=10,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}),
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
len(vectorizer.vocabulary_)

10441

In [ ]:
vectorizer.vocabulary_

{'wondering': 10274,
 'enlighten': 3611,
 'car': 1923,
 'saw': 8328,
 'day': 2890,
 'door': 3319,
 'sports': 8900,
 'looked': 5797,
 'late': 5568,
 '60s': 546,
 'early': 3431,
 '70s': 597,
 'called': 1877,
 'doors': 3320,
 'really': 7761,
 'small': 8745,
 'addition': 830,
 'bumper': 1808,
 'separate': 8492,
 'rest': 8040,
 'body': 1645,
 'know': 5499,
 'model': 6234,
 'engine': 3595,
 'specs': 8863,
 'years': 10393,
 'production': 7433,
 'history': 4722,
 'info': 5051,
 'looking': 5798,
 'mail': 5895,
 'fair': 3893,
 'number': 6581,
 'brave': 1727,
 'souls': 8818,
 'upgraded': 9858,
 'si': 8630,
 'clock': 2204,
 'oscillator': 6782,
 'shared': 8557,
 'experiences': 3798,
 'poll': 7207,
 'send': 8473,
 'brief': 1748,
 'message': 6100,
 'detailing': 3085,
 'procedure': 7414,
 'speed': 8869,
 'cpu': 2711,
 'rated': 7715,
 'add': 827,
 'cards': 1930,
 'adapters': 824,
 'heat': 4650,
 'hour': 4795,
 'usage': 9876,
 'floppy': 4085,
 'disk': 3229,
 'functionality': 4238,
 '800': 624,
 'floppie

In [ ]:
X_train = vectorizer.fit_transform(train.data)

In [ ]:
X_train.shape

(11314, 10441)

In [ ]:
n = 343
print('Topic = {0}\n'.format(train.target_names[train.target[n]]))
print(train.data[n])

Topic = sci.space


In fact, you probably want to avoid US Government anything for such a
project.  The pricetag is invariably too high, either in money or in
hassles.

The important thing to realize here is that the big cost of getting to
the Moon is getting into low Earth orbit.  Everything else is practically
down in the noise.  The only part of getting to the Moon that poses any
new problems, beyond what you face in low orbit, is the last 10km --
the actual landing -- and that is not immensely difficult.  Of course,
you *can* spend sagadollars (saga- is the metric prefix for beelyuns
and beelyuns) on things other than the launches, but you don't have to.

The major component of any realistic plan to go to the Moon cheaply (for
more than a brief visit, at least) is low-cost transport to Earth orbit.
For what it costs to launch one Shuttle or two Titan IVs, you can develop
a new launch system that will be considerably cheaper.  (Delta Clipper
might be a bit more expensive than this, 

In [ ]:
from tqdm import tqdm
def lda(n, X, W , a, b):
    n_kw = np.zeros((n, X.shape[1]))
    n_dk = np.zeros((X.shape[0], n))
    n_k = np.zeros(n)
    docs, words = X.nonzero()
    tags = np.random.choice(n, len(docs))
    for w,d,t in zip(words, docs, tags):
        n_kw[t,w] += 1
        n_dk[d,t] += 1
        n_k[t] +=1
    for i in tqdm(range(W)):
        for j in range(len(docs)):
            t=tags[j]
            n_kw[t,words[j]] -= 1
            n_dk[docs[j], t] -=1
            n_k[t] -=1
            p=(n_dk[docs[j], :] + a)*(n_kw[:,words[j]] + b[words[j]]) /(n_k + b.sum())
            tags[j] = np.random.choice(np.arange(n), p = p / p.sum())
            n_kw[tags[j], words[j]] += 1
            n_dk[ docs[j], tags[j]] += 1
            n_k[tags[j]] += 1
    return n_kw, n_dk, n_k, tags

In [ ]:
n=20
n_kw, n_dk, n_k, tags= lda(n, X_train, 50, 1*np.ones(n), 1*np.ones(X_train.shape[1]))

100%|██████████| 50/50 [27:35<00:00, 33.11s/it]


In [ ]:
answer=np.argsort(nkw, axis=1)[:, -10:]
for i in range(20):
    matrix = np.zeros((1, X_train.shape[1]))
    for j in answer[i]:
        matrix[0, j] = 1
    print('Tag {}:\t{}'.format(i+1, '\t'.join(vectorizer.inverse_transform(matrix)[0])))
    
#Можно заметить, что много слов относятся к конкретным темам. Например, упоминание слов "Израиль", "евреи" или "война"
#позволяют предположить, что речь идёт о Ближнем Востоке. Этот набор в наибольшей степени представляет следующие темы: 
#sci.space, talk.politics.guns, talk.politics.mideast, sci.crypt, misc.forsale, rec.autos.

Tag 1:	does	just	know	looking	number	ok	right	thank	time	true
Tag 2:	believe	does	fact	god	jesus	life	people	point	say	true
Tag 3:	chip	clipper	encryption	government	key	keys	public	use	used	using
Tag 4:	cost	earth	high	large	low	nasa	new	program	research	space
Tag 5:	game	games	good	league	play	players	season	team	win	year
Tag 6:	address	ca	com	edu	email	information	list	mail	send	university
Tag 7:	does	don	good	just	know	let	like	post	thanks	want
Tag 8:	file	files	problem	program	run	thanks	use	using	window	windows
Tag 9:	best	edu	interested	mail	new	offer	price	sale	sell	used
Tag 10:	day	did	got	just	old	said	time	told	took	went
Tag 11:	better	don	good	just	like	long	probably	think	time	ve
Tag 12:	believe	did	didn	don	gas	just	know	like	people	think
Tag 13:	don	just	like	new	right	think	use	used	ve	way
Tag 14:	card	disk	does	drive	pc	problem	thanks	use	video	work
Tag 15:	banks	cause	edu	gordon	medical	pitt	problems	soon	usually	years
Tag 16:	10	11	12	14	15	16	18	20	24	25
Tag 17:	gov